# 第1回：MedMNISTによる深層学習モデル入門

## 学習目標

- Google Colab の扱いを学ぶ
- 医用画像データセット MedMNIST の扱いを学ぶ
- 畳み込みニューラルネットワーク（CNN）の学習プロセスを理解する

---
## ようこそ！医療AI開発の世界へ

この講義では、**実際に動くコード**を通じて深層学習の基礎を学びます。
プログラミングの経験がなくても大丈夫。セルを順番に実行していくだけで、
あなた自身の手で「病理画像を診断するAI」を構築できます。

## Google Colabとは

Google Colaboratory（通称: Colab）は、ブラウザ上でPythonを実行できる無料の環境です。

**Colabの特徴:**
- インストール不要：ブラウザさえあれば、どのPCからでも利用可能
- GPU無料利用：AIの学習に必要な高性能計算を無料で実行できる
- 自動保存：作業内容はGoogleドライブに自動保存される

**注意事項:**
- 一定時間（約90分）操作しないとセッションが切断されます
- 切断されても、上から順にセルを再実行すれば復旧できます
- 講義中は定期的にセルを実行して、セッション切れを防ぎましょう

## この講義の進め方

1. **コードセルを上から順に実行**: `Shift + Enter` でセルを実行します
2. **出力を観察**: 各セルの実行結果を確認しながら進めます
3. **練習問題に挑戦**: パラメータを変更して、結果の変化を観察します
4. **考察課題を考える**: 「なぜこうなるのか」を自分の言葉で説明してみましょう

では、始めましょう！

---
## MedMNISTとは

MedMNISTは、医用画像を28×28ピクセルに標準化した教育用データセットです。病理画像、皮膚画像、胸部X線など、10種類以上の医療画像が含まれています。

手書き数字認識で有名な「MNIST」の医療版と考えてください。MNISTが数字の「0〜9」を分類するように、MedMNISTは医療画像を「正常・異常」や「疾患の種類」で分類します。

## CNN（畳み込みニューラルネットワーク）とは

CNNは、画像認識に特化したニューラルネットワークです。画像の局所的な特徴（エッジや模様）を自動的に学習し、分類に活用します。

人間が絵を見るとき、まず線や色を認識し、次に形を把握し、最後に「これは猫だ」と判断します。CNNも同様に、単純な特徴から複雑な特徴へと段階的に認識を深めていきます。

In [ ]:
# 環境セットアップ
!pip install medmnist -q
import sys, os
!rm -rf /tmp/MedMNIST-Exercise
!git clone https://github.com/kshimoji8/MedMNIST-Exercise.git /tmp/MedMNIST-Exercise -q
sys.path.insert(0, '/tmp/MedMNIST-Exercise')
sys.modules.pop('exercise_logic', None)
import exercise_logic
exercise_logic.initialize_environment()
print("✓ セットアップが完了しました。")

# --- 病理データのロード ---
(x_train, y_train), (x_test, y_test), info = exercise_logic.load_and_preprocess('pathmnist')
print(f"✓ データロード完了: 訓練データ {x_train.shape}, ラベル {y_train.shape}")

---
## CNNモデルの構成要素

- **畳み込み層（Conv2D）**: 画像からエッジや模様などの特徴を抽出します。虫眼鏡で画像の一部分を見ながら「ここに線がある」「ここに丸がある」と特徴を見つけていくイメージです。

- **プーリング層（MaxPooling2D）**: 特徴マップのサイズを縮小し、重要な情報を残します。写真を縮小しても何が写っているかわかるように、細かい情報を捨てて本質的な特徴だけを残します。

- **全結合層（Dense）**: 抽出された特徴を元に、最終的な分類を行います。「エッジがこう並んでいて、この模様があるから、これは病変だ」と総合判断する部分です。

In [ ]:
# シンプルなCNNモデルを構築
model = exercise_logic.build_model(
    input_shape=(28, 28, 3), 
    num_classes=len(info['label']), 
    model_type='simple'
)

model.summary()

---
## 学習プロセスの用語

- **エポック（epochs）**: 全データを何回繰り返し学習するかを指定します。教科書を1回読むより3回読んだ方が覚えられるように、繰り返し学習することで精度が向上します。

- **バッチサイズ（batch_size）**: 一度に処理するデータの数です。単語帳を1枚ずつめくるか、10枚まとめて覚えるかの違いです。大きいほど学習が安定しますが、メモリを多く使います。

- **検証データ（validation_split）**: 学習中に性能を確認するために分けておくデータの割合です。模擬試験のようなもので、本番（テストデータ）の前に実力を確認できます。

In [ ]:
# 学習の実行
history = model.fit(
    x_train, y_train, 
    epochs=10, 
    validation_split=0.1, 
    batch_size=128
)

---
## 評価指標の見方

- **学習曲線**: 損失（loss）が下がり、精度（accuracy）が上がれば学習が進んでいます。ただし、訓練データの精度だけ上がって検証データの精度が上がらない場合は「過学習」の兆候です。丸暗記で応用が利かない状態と同じです。

- **混同行列**: 実際のラベルと予測ラベルの対応を表にしたものです。対角線上の数値が多いほど正確です。「猫を犬と間違えた回数」「犬を猫と間違えた回数」などが一目でわかります。

In [ ]:
# 評価と分析
exercise_logic.plot_history(history)
exercise_logic.show_evaluation_reports(model, x_test, y_test, info['label'])

---
## CNNの中を覗いてみよう：特徴マップの可視化

CNNが画像をどのように「見ている」かを確認してみましょう。

畳み込み層を通過した後の画像（**特徴マップ**）を可視化すると、CNNが抽出している特徴を直接観察できます。

- 第1層では、エッジや色の境界など単純な特徴を検出
- 層が深くなるにつれて、より複雑なパターンを認識

In [ ]:
# テスト画像の特徴マップを可視化
sample_image = x_test[0]

# 第1畳み込み層の特徴マップを表示
exercise_logic.visualize_feature_maps(model, sample_image)

### CNNの処理の流れ

入力画像がどのように処理されて最終的な予測に至るか、一連の流れを確認しましょう。

In [ ]:
# CNNの処理の流れを可視化（入力→特徴抽出→予測）
exercise_logic.visualize_cnn_flow(model, x_test[0], info['label'])

### 他の画像でも試してみよう

`x_test[0]` の `0` を別の数字（例: `5`, `10`, `100`）に変えて、異なる画像での特徴マップを観察してください。

In [ ]:
# 別の画像で試す（数字を変えてみてください）
exercise_logic.visualize_cnn_flow(model, x_test[5], info['label'])

---
## 練習問題

### 練習1: エポック数を変更してみよう

上の学習コードで `epochs=10` を `epochs=5` や `epochs=20` に変更して、精度がどう変わるか観察してください。

```python
history = model.fit(
    x_train, y_train, 
    epochs=20,  # ← ここを変更
    validation_split=0.1, 
    batch_size=128
)
```

**観察ポイント:**
- エポック数を増やすと精度は上がり続けますか？
- 訓練精度と検証精度の差はどう変化しますか？

### 練習2: バッチサイズを変更してみよう

`batch_size=128` を `batch_size=32` や `batch_size=256` に変更して、学習の様子を比較してください。

**観察ポイント:**
- 学習速度（1エポックあたりの時間）はどう変わりますか？
- 学習曲線の滑らかさに違いはありますか？

---
## 考察課題

以下の点について考えてみましょう：

1. **過学習の兆候**: 学習曲線で「訓練精度は上がり続けるのに、検証精度が下がり始める」現象が見られた場合、どう対処すべきでしょうか？

2. **医療画像への応用**: 病理画像の分類にCNNを使う場合、「98%の精度」は十分でしょうか？残り2%の誤診が持つ意味について考えてください。

3. **データの重要性**: 訓練データに偏りがある場合（例：特定の病院のデータのみ）、モデルの性能にどのような影響がありますか？

---
## まとめ

本講義で学んだ内容：

- **MedMNIST**: 医用画像を28×28ピクセルに標準化した教育用データセット
- **CNN**: 画像の特徴を自動的に学習し、分類を行うニューラルネットワーク
- **畳み込み層**: 画像から局所的な特徴（エッジ、模様）を抽出
- **プーリング層**: 特徴マップを縮小し、重要な情報を保持
- **学習プロセス**: エポック数、バッチサイズ、検証データの役割
- **評価指標**: 学習曲線と混同行列による性能評価

次回は「転移学習」と「クラス不均衡への対処」を学びます。

---
## 考察課題の回答例

以下は考察課題に対する回答の一例です。これが唯一の正解ではなく、議論の出発点として活用してください。

### 1. 過学習の兆候と対処法

過学習（オーバーフィッティング）が疑われる場合の代表的な対処法：

- **早期終了（Early Stopping）**: 検証損失（validation loss）や監視指標が一定期間改善しなくなったら（patienceを設けて）学習を停止する
- **ドロップアウト**: 学習中にユニット（または結合）を確率的に無効化し、特定の特徴への過度な依存を抑える
- **データ拡張**: 回転・反転・拡大縮小などで入力の多様性を増やし、汎化性能を高める（医療画像では不適切な拡張が診断所見を壊し得る点に注意）
- **正則化**: 重み減衰（L2/weight decay）などでモデル複雑性にペナルティを与え、過度な当てはまりを抑制する

### 2. 医療AIの「精度」の意味

「正解率98%」は一見高いが、医療ではそれだけでは不十分なことがある：

- 1000件の判定で誤りが2%なら、単純計算で20件の誤りが起こり得る
- 医療では、正解率（accuracy）だけでなく、**感度・特異度、陽性的中率/陰性的中率（有病率に依存）、ROC/AUC**などの指標が重要になる
- がんの見逃し（偽陰性）は治療開始の遅れにつながり得る一方、偽陽性は不要な検査・心理的負担・医療資源の消費につながり得る
- 実運用では「意図された使用（診断支援、トリアージ等）」に応じて、重視すべき誤り（偽陰性/偽陽性）と閾値設計を明確にする

### 3. データの偏り（バイアス）と汎化性能

特定施設（単施設）のデータのみで学習すると：

- 患者背景や紹介バイアス、撮影プロトコル、ラベル付けの流儀などに引きずられ、他施設で性能が落ちる可能性がある
- 機器（メーカー、撮影条件、再構成条件）の差で分布が変わり、性能が変動し得る
- **外部検証（別施設・別地域）**や、**時系列での検証（過去→現在）**を含めた評価が重要になる

---
## 発展的な学習（技術的詳細に興味のある方へ）

この講義では、技術的な詳細を `exercise_logic.py` に分離しています。
より深く学びたい方は、以下の関数のソースコードを参照してください。

### この講義で使用した主要関数

| 関数名 | 機能 | 技術的なポイント |
|--------|------|------------------|
| `initialize_environment()` | 環境セットアップ | Colab/Local判定、GPU設定 |
| `load_and_preprocess()` | データ読み込み・前処理 | MedMNISTの構造、正規化、チャンネル変換 |
| `build_model()` | CNNモデル構築 | Conv2D、MaxPooling、活性化関数の役割 |
| `plot_history()` | 学習曲線の描画 | 過学習の診断方法 |
| `show_evaluation_reports()` | 評価指標の表示 | 混同行列、Precision/Recall/F1の意味 |
| `visualize_feature_maps()` | 特徴マップ可視化 | Functional APIによる中間出力取得 |
| `visualize_cnn_flow()` | CNN処理フロー可視化 | 入力から予測までの流れ |

### ソースコードの参照方法

`exercise_logic.py` はGitHubリポジトリで公開しています：

https://github.com/kshimoji8/MedMNIST-Exercise/blob/main/exercise_logic.py

各関数には詳細な技術解説をdocstring（関数冒頭のコメント）として記載しています。